In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

In [4]:
track_history = defaultdict(lambda: [])
model = YOLO("yolov8n.pt")
names = model.model.names

In [6]:
print(names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [32]:
video_path = "/path/to/video/file.mp4"
cap = cv2.VideoCapture(2)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

result = cv2.VideoWriter("object_tracking.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):
                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))
                if len(track) > 30:
                    track.pop(0)

                # Plot tracks
                points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

result.release()
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [11]:

# Load the YOLOv8 model
model = YOLO('yolov8x-pose.pt')

# Open the video file
video_path = "path/to/video.mp4"
cap = cv2.VideoCapture(1)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True,)

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        print(results[0])

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

[ WARN:0@406.575] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@406.575] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


In [27]:
from collections import defaultdict

import cv2
import numpy as np

from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
video_path = "path/to/video.mp4"
cap = cv2.VideoCapture(2)

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 1 tv, 2.0ms
Speed: 0.9ms preprocess, 2.0ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 2.2ms
Speed: 0.8ms preprocess, 2.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 1.9ms
Speed: 0.8ms preprocess, 1.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 2.8ms
Speed: 0.8ms preprocess, 2.8ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 3.3ms
Speed: 1.0ms preprocess, 3.3ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 3.0ms
Speed: 0.7ms preprocess, 3.0ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 2.8ms
Speed: 0.8ms preprocess, 2.8ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tv, 2.1ms
Speed: 1.0ms preprocess, 2.1ms inference, 0.5ms postproc

AttributeError: 'NoneType' object has no attribute 'int'

In [7]:
import cv2
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8l.pt')

# Open the video file
video_path = "path/to/video.mp4"
cap = cv2.VideoCapture(0)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True,conf=0.6, iou=0.5)

        x = results[0].boxes
        print(x)

        # Visualize the results on the frame
        annotated_frame = results[0].plot(show=True)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 56.5ms
Speed: 0.8ms preprocess, 56.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9631])
data: tensor([[  0.0000,  92.4592, 476.7337, 479.6353,   1.0000,   0.9631,   0.0000]])
id: tensor([1.])
is_track: True
orig_shape: (480, 640)
shape: torch.Size([1, 7])
xywh: tensor([[238.3668, 286.0472, 476.7337, 387.1761]])
xywhn: tensor([[0.3724, 0.5959, 0.7449, 0.8066]])
xyxy: tensor([[  0.0000,  92.4592, 476.7337, 479.6353]])
xyxyn: tensor([[0.0000, 0.1926, 0.7449, 0.9992]])

0: 480x640 1 person, 10.9ms
Speed: 1.1ms preprocess, 10.9ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9639])
data: tensor([[9.3617e-02, 9.3208e+01, 4.7552e+02, 4.7965e+02, 1.0000e+00, 9.6389e-01, 0.0000e+00]])
id: tensor([1.])
is_track: True
orig_shape: (480, 640)
shape: torch

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/adyansh/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/adyansh/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/adyansh/snap/code/common/.cache/gio-modules/libgiolibproxy.so


0: 480x640 1 person, 11.4ms
Speed: 1.1ms preprocess, 11.4ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9638])
data: tensor([[  0.0000,  94.2687, 475.9552, 479.5849,   1.0000,   0.9638,   0.0000]])
id: tensor([1.])
is_track: True
orig_shape: (480, 640)
shape: torch.Size([1, 7])
xywh: tensor([[237.9776, 286.9268, 475.9552, 385.3162]])
xywhn: tensor([[0.3718, 0.5978, 0.7437, 0.8027]])
xyxy: tensor([[  0.0000,  94.2687, 475.9552, 479.5849]])
xyxyn: tensor([[0.0000, 0.1964, 0.7437, 0.9991]])

0: 480x640 1 person, 10.8ms
Speed: 1.8ms preprocess, 10.8ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9656])
data: tensor([[1.1171e-01, 9.4125e+01, 4.7569e+02, 4.7962e+02, 1.0000e+00, 9.6563e-01, 0.0000e+00]])
id: tensor([1.])
is_track: True
orig_shape: (480, 640)
shape: torch.

eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/adyansh/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/adyansh/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE



0: 480x640 1 person, 11.0ms
Speed: 1.4ms preprocess, 11.0ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9671])
data: tensor([[  0.0000, 103.2943, 473.4943, 473.9912,   1.0000,   0.9671,   0.0000]])
id: tensor([1.])
is_track: True
orig_shape: (480, 640)
shape: torch.Size([1, 7])
xywh: tensor([[236.7471, 288.6428, 473.4943, 370.6970]])
xywhn: tensor([[0.3699, 0.6013, 0.7398, 0.7723]])
xyxy: tensor([[  0.0000, 103.2943, 473.4943, 473.9912]])
xyxyn: tensor([[0.0000, 0.2152, 0.7398, 0.9875]])

0: 480x640 1 person, 10.6ms
Speed: 1.0ms preprocess, 10.6ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9653])
data: tensor([[  0.0000, 109.2134, 473.6267, 473.8888,   1.0000,   0.9653,   0.0000]])
id: tensor([1.])
is_track: True
orig_shape: (480, 640)
shape: torch.Size([1, 7])


...

0: 480x640 1 person, 14.9ms
Speed: 1.1ms preprocess, 14.9ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9629])
data: tensor([[1.4577e-01, 1.2274e+02, 4.4809e+02, 4.7924e+02, 1.0000e+00, 9.6290e-01, 0.0000e+00]])
id: tensor([1.])
is_track: True
orig_shape: (480, 640)
shape: torch.Size([1, 7])
xywh: tensor([[224.1187, 300.9936, 447.9459, 356.5022]])
xywhn: tensor([[0.3502, 0.6271, 0.6999, 0.7427]])
xyxy: tensor([[1.4577e-01, 1.2274e+02, 4.4809e+02, 4.7924e+02]])
xyxyn: tensor([[2.2776e-04, 2.5571e-01, 7.0014e-01, 9.9843e-01]])

0: 480x640 1 person, 14.5ms
Speed: 1.0ms preprocess, 14.5ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9638])
data: tensor([[  1.1179, 122.0621, 448.1207, 479.7339,   1.0000,   0.9638,   0.0000]])
id: tensor([1.])
is_track: True
orig_shape: 

eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/adyansh/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/adyansh/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE



0: 480x640 1 person, 14.7ms
Speed: 1.0ms preprocess, 14.7ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9632])
data: tensor([[  0.0000, 120.5587, 445.0332, 479.7377,   1.0000,   0.9632,   0.0000]])
id: tensor([1.])
is_track: True
orig_shape: (480, 640)
shape: torch.Size([1, 7])
xywh: tensor([[222.5166, 300.1482, 445.0332, 359.1790]])
xywhn: tensor([[0.3477, 0.6253, 0.6954, 0.7483]])
xyxy: tensor([[  0.0000, 120.5587, 445.0332, 479.7377]])
xyxyn: tensor([[0.0000, 0.2512, 0.6954, 0.9995]])


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/adyansh/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/adyansh/snap/code/common/.cache/gio-modules/libgiolibproxy.so


eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
